# Bokit tutorial

This notebook provides an overview of the bokit Python API to create and manipulate Boolean functions.

In [1]:
import bokit

# Variables and expressions

Boolean variables are identified by positive integers used as unique identifiers (UIDs).
Note that the UIDs can be any positive integer (including 0). It is OK to skip some values,
but it may have a cost of the readability and memory usage of states and patterns.


Variables don't do much by themselves, but they can be used to define Boolean rules
using the ``&``,``|`` and ``~`` operators. If the obects ``a``, ``b``, and ``c`` represent
Boolean variables, then the expression ``a & (b | ~c)`` is a Boolean expression which is
true if the variable ``a`` is true and either ``b`` is true or ``c`` is false.

In [2]:
# We can create some variables associated to integer UIDs
a,b,c,d,e = [ bokit.Variable(i) for i in range(5) ]

# Printing a variable gives a generic name based on the UID
print(a)

# We can retrieve the internal integer UID (but not change it)
a.uid()

_0_


0

In [3]:
# Variables can be combined into expressions
expr = a | (b & ~c)
expr

__0__ | __1__ & !__2__

In [4]:
expr2 = bokit.Expr("_1 & !(_1 | !_2)")

# States

A state is a set of active variables, which can be used to evaluate a Boolean expression.
All variables missing from the state are considered as inactive.

Note that as states do not keep track of the list of variables, the set of inactive variables
depends on additional external information.

We can create an empty set and use associated functions to add or remove active variables
(using variable objects defined above) from the set.
We can also parse a textual representation directly into a state object.

In [5]:
# Create a default state (all variables are inactive)
state = bokit.State()

# Enable some variables in a state
state.activate(d)
state.activate(b)
print(state)

# Add and remove more variables
state.activate(a)
state.disable(b)
print(state)

0101
1001


In [6]:
# A state can also be created by parsing a string
state1 = bokit.State("101")
state2 = bokit.State("111")
state3 = bokit.State("110")

state2.activate(e)
state1.disable(a)

In [7]:
# Printing a state gives a text representation that can be parsed
# Note that an empty state gives an empty string, and that this representation
# stops at the active variable with the largest UID
print(state1)
print(state2)

001
11101


In [8]:
# Here again, the raw representation gives more details
state

State { active: VarSet({0, 3}) }

In [9]:
# States can be used to evaluate an expression
print(expr.eval(state))
print(expr.eval(state1))
print(expr.eval(state2))
print(expr.eval(state3))

True
False
True
True


# Patterns

A pattern defines two sets set of active and inactive variables.
All other variables are considered as "free".

A pattern represents all the states which satisfy the constraints, forming an hypercube with the free variables.
It can be turned into an expression, which is a conjunction of the included constraints.
This expression will be true for all states in the pattern and false for all other states. 

Note that like the states, patterns do not carry information about the number of variables.
The set of free variables and the number of states contained in a pattern depend on additional information.

In [10]:
# Create patterns described as strings
pattern1 = bokit.Pattern("01-")
pattern2 = bokit.Pattern("110")

In [11]:
# Turn a pattern into an expression
expr2 = bokit.Expr(pattern1)
expr2

__1__ & !__0__

In [12]:
# A pattern can be evaluated just like an expression
# This amounts to testing if the pattern contains the given state
print(expr2.eval(state1))
print(expr2.eval(state2))
print(expr2.eval(state3))

False
False
False


In [13]:
print(pattern1.contains_state(state1))
print(pattern1.contains_state(state2))
print(pattern1.contains_state(state3))

False
False
False


# Prime implicants

A pattern is an implicant of an exression, if this expression is true for all states in the pattern.
An implicant is prime if it is not contained in a more general implicant.
The prime implicants of an expression provide a normalized reresentation of the expression, and can
be used to optimize some computations.

Bokit provides an efficient method to compute all prime implicants of an expression.

In [14]:
s_implicants = """
11--1-0
--01--1
"""

implicants = bokit.Implicants(s_implicants)
print(f"Implicants:\n{implicants}")

primes = bokit.Primes(s_implicants)
print(f"Primes:\n{primes}")

Implicants:
11--1-0
--01--1

Primes:
11--1-0
--01--1
11011



In [15]:
# Turn an expression into a set of prime implicants
pis = bokit.Primes(expr)
pis

1
-10

In [16]:
# The set of prime imnplicants can be also used to evaluate a function
print(pis.eval(state1))
print(pis.eval(state2))
print(pis.eval(state3))

False
True
True


In [17]:
# It further enables fuzzy evaluation over a pattern
pis.satisfiable_in_pattern(pattern1), pis.satisfiable_in_pattern(pattern2)

(True, True)

# Variable groups: human-friendlier display with named variables

In [18]:
vs = bokit.VarSpace()

In [19]:
v = vs.provide("test")
k = vs.provide("other")
v2 = vs.provide("other:1")

In [20]:
mye = vs.parse_expression("test | ~other")

In [21]:
print( mye )
print(vs.display(mye))

!__1__ | __0__
!other:0 | test


In [22]:
vs.display(mye)

'!other:0 | test'

In [23]:
vs.rename('test', 'valid')
vs.set_name(k, 'last')

Variable(1)

In [24]:
vs.display(mye)

'!last:0 | valid'

In [25]:
try:
    vs.parse_expression("test | ~other & pipo")
    print("This should have failed!!")
except:
    print("Error while parsing as expected")

Error while parsing as expected


In [26]:
vs.set_auto_extend(True)
e = vs.parse_expression("test | ~(plop | alternate & test)")
vs.set_auto_extend(False)
vs.display(e)

'test | !(plop | test & alternate)'

# Multi-valued variables

While raw variables are purely Boolean, variable collections let us create ordered groups of associated variables, that can be used to account for multi-valued variables.

In [27]:
v = vs['test']
vs.display(v)

'test'

In [28]:
# Add an associated variable
v1 = vs.provide("test:1")
vs.display(v1)

'test:1'

In [29]:
# The original variable is now associated to order 0
vs.display(v)

'test:0'

In [30]:
# Add an associated variable to an existing variable
v2 = vs.provide("test:2")
vs.display(v2)

'test:2'

In [31]:
# Renaming affects all variables of the group
vs.rename('test', 'another')
vs.display(v), vs.display(v1), vs.display(v2)

('another:0', 'another:1', 'another:2')